| [![back](../../media/navigation/back.png)](../../exercises/tissue-classification-segmentation/ex2.html) | [![home](../../media/navigation/home.png)](../../index.html) | [![next](../../media/navigation/next.png)](../../exercises/tissue-classification-segmentation/ex4.html) |
| :---: | :---: | :---: |
| Ex 4.2: Threshold-based tissue segmentation | • | Ex 4.4: Semi-automatic segmentation using SAM |

# 4. Tissue segmentation and classification

## 4.3 Using a N-classes pixel classifier

In the last exercise, we discovered a simple way to extract our objects using a simple thresholder, but they are very limited in terms of what they can see (only based on intensity) and we are limited to two classes (true/false, +/-, FG/BG, ...)

We would like to have a way to segment areas in our tissue based on features more subtle than just the intensity.
We would also like to have the possibility to classify more than two types of tissue.


### 4.3.1 Epithelium area in colorectal cancer

#### Goals:

- The goal of this exercise is to measure the area of epithelium in images of colorectal cancers.
- To do so, you will need to:
    - Find the polygon corresponding to the whole tissue and give it the `Colon` classification using a thresholder.
    - Make a training image from a set of representative chunks of image.
    - Train a pixel classifier able to differentiate the epithelium from the rest of the tissue.
    - Apply the pixel classifier to find epithelium detections within the colon annotations.
    - Export the measurements to a TSV.

#### Required data:

| **Folder** | Description | Location | License |
| --- | --- | --- | --- |
| Colorectal Cancer IHC HE | Whole slide images of colorectal cancer stained with hematoxylin and eosin | [DOI: 10.18710/DIGQGQ](https://dataverse.no/dataset.xhtml?persistentId=doi:10.18710/DIGQGQ) | CC0 1.0 |

### A. Locate the tissue on the slides

- Before starting this step, make sure that you have the `Tumor` class available in your class list. It is part of the built-in classes of QuPath so it should already be present for everybody.
- Using the protocol described in [Ex 2: Threshold-based tissue segmentation](../../exercises/tissue-classification-segmentation/ex2.html#A.-Locate-the-lungs-tissue), create the threshold classifier segmenting the tissue on your slides. Make sure that you split objects into separate polygons so you can filter the polygons by area and give a name to the different parts so you can find in the final TSV which line corresponds to which object.
- Using the workflow recorder, apply the thresholder to all the images of your dataset and clean up the results where it's required:
    - Remove the undesired parts (bubbles, folded tissue, ...)
    - Merge fragmented tissue into unique polygons
    - Give a name to each chunk to recognize them later (In the annotation list: right-click > Set properties > Name)
- At this point, each image that you have should contain several polygons classified as `Tumor`.
- If you gave names to your annotations but they don't show up in the viewer, try to click on the ![QP toggle name](../../media/qp-icons/show-names.png) "toggle names display" button.

![segmented colon tumor](../../media/tissue-classification/segmented-colon-tumor.png)

### B. Create a training image

- The pixel classifier that we will use later is based on a machine learning algorithm, so we need to show it a representative sample of data.
- We can't simply search through our whole project and hope to find an image representative of the entire dataset.
- To address this need, we will create a new image being a patchwork of representative chunks selected throughout our project.
- In order to do that, make a checklist of the different textures that you can find your your images, for example:
    - Where the nuclei are very dense
    - Where there is almost no nuclei
    - Where you have small/huge crypts
    - Where the staining is lighter/darker than usual
    - ...
- Once you identified all the textures, for each of these textures:
    - Find an example of area where this texture is present
    - Activate your [QP rectangle](../../media/qp-icons/rectangle-selection.png) rectangle selection tool and make a rectangle around this example
    - Give the `Region*` class to this newly created rectangle

**Note:**

Don't forget to include a couple of "normal"/"clean" areas in your `Region*` rectangles, it would be a shame if the classifier only behaved well on unusual cases!